# Estimation of MET cross section with Madgraph 3.6.0

In [5]:
# !pip install numpy pandas sympy matplotlib 
# !wget https://launchpad.net/mg5amcnlo/3.0/3.6.x/+download/MG5_beta_v3.6.0.tgz
# !sudo apt-get update

In [6]:
import pandas as pd

# Criação da tabela de referência de partículas
particle_data = {
    "PID": [11, -11, 13, -13, 22, 12, -12, 14, -14, 16, -16, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 21, 23, 24, -24],
    "Name": [
        "Electron", "Positron", "Muon", "Anti-Muon", "Photon",
        "Electron Neutrino", "Electron Anti-Neutrino", "Muon Neutrino", "Muon Anti-Neutrino",
        "Tau Neutrino", "Tau Anti-Neutrino", "Down Quark", "Anti-Down Quark",
        "Up Quark", "Anti-Up Quark", "Strange Quark", "Anti-Strange Quark",
        "Charm Quark", "Anti-Charm Quark", "Bottom Quark", "Anti-Bottom Quark",
        "Gluon", "Z Boson", "W+ Boson", "W- Boson"
    ],
    "Symbol": [
        "e-", "e+", "mu-", "mu+", "a",
        "ve", "ve~", "vm", "vm~",
        "vt", "vt~", "d", "d~",
        "u", "u~", "s", "s~",
        "c", "c~", "b", "b~",
        "g", "z", "w+", "w-"
    ]
}

# Criação do DataFrame com as informações de partículas
particle_df = pd.DataFrame(particle_data)
particle_df.to_csv('particle_names.csv', index=False)
print(particle_df)


    PID                    Name Symbol
0    11                Electron     e-
1   -11                Positron     e+
2    13                    Muon    mu-
3   -13               Anti-Muon    mu+
4    22                  Photon      a
5    12       Electron Neutrino     ve
6   -12  Electron Anti-Neutrino    ve~
7    14           Muon Neutrino     vm
8   -14      Muon Anti-Neutrino    vm~
9    16            Tau Neutrino     vt
10  -16       Tau Anti-Neutrino    vt~
11    1              Down Quark      d
12   -1         Anti-Down Quark     d~
13    2                Up Quark      u
14   -2           Anti-Up Quark     u~
15    3           Strange Quark      s
16   -3      Anti-Strange Quark     s~
17    4             Charm Quark      c
18   -4        Anti-Charm Quark     c~
19    5            Bottom Quark      b
20   -5       Anti-Bottom Quark     b~
21   21                   Gluon      g
22   23                 Z Boson      z
23   24                W+ Boson     w+
24  -24                W-

In [7]:
particle_df = pd.read_csv('particle_names.csv')


In [8]:
import subprocess
import os
import pandas as pd
import mplhep as hep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def run_madgraph(madgraph_process: str, event_name: str, run_parameters: dict):
    # Definir o diretório onde o MadGraph está instalado e o diretório de trabalho
    particle_df = pd.read_csv('particle_names.csv')
    current_path = os.getcwd()

    madgraph_path = f"{current_path}/MG5_aMC_v3_6_0/bin/mg5_aMC"
    working_dir = f"{current_path}/workdir"

    process_dir = os.path.join(working_dir, event_name)
    # Crie um arquivo de comando do MadGraph para geração de eventos
    def run_command_mg5(command: str):
        command_file = os.path.join(working_dir, "process_command.txt")
        
        with open(command_file, "w") as f:
            f.write(command)
            
        # Execute o MadGraph usando o subprocess
        try:
            subprocess.run([madgraph_path, command_file], cwd=working_dir, check=True);
            print("MadGraph executado com sucesso.")
        except subprocess.CalledProcessError as e:
            print("Erro ao executar MadGraph:", e)

    run_command_mg5(f""" generate {madgraph_process}
            output {event_name}""")

    # Path to param_card.dat
    param_card_path = os.path.join(process_dir, "Cards", "param_card.dat")

    # Function to edit parameters in param_card.dat
    def edit_param_card(param_card_path, parameters):
        with open(param_card_path, 'r') as file:
            lines = file.readlines()

        # Modify parameters as specified in 'parameters' dictionary
        with open(param_card_path, 'w') as file:
            for line in lines:
                for param, value in parameters.items():
                    # Replace the value for the given parameter name in the param_card file
                    if param in line:
                        # Preserve line formatting by replacing the value at the correct position
                        parts = line.split()
                        if len(parts) >= 2:
                            parts[-1] = str(value)  # Replace the last element with new value
                            line = " ".join(parts) + "\n"
                file.write(line)

    # Define parameters to modify: keys are param names, values are the new values
    # Adjust these parameter names based on your process and param_card format
    # parameters_to_modify = {
    #     "MASS 6": 172.5,     # Modify the top quark mass, for example
    #     "WIDTH 6": 1.32,     # Modify the top quark width
    # }
    # Edit the param_card.dat with specified parameters
    #edit_param_card(param_card_path, parameters_to_modify)
    #print("Parameters in param_card.dat have been updated.")

    # Path to run_card.dat
    run_card_path = os.path.join(process_dir, "Cards", "run_card.dat")

    # Function to edit parameters in run_card.dat
    def edit_run_card(run_card_path, run_parameters):
        with open(run_card_path, 'r') as file:
            lines = file.readlines()

        # Modify parameters as specified in 'parameters' dictionary
        with open(run_card_path, 'w') as file:
            for line in lines:
                for param, value in run_parameters.items():
                    # Replace the value for the given parameter name in the run_card file
                    if param in line:
                        parts = line.split("=")
                        if len(parts) >= 2:
                            parts[0] = f"     {value}      "
                            line = "=".join(parts)
                        print("new line modified: ", line)

                file.write(line)

    # Define parameters to modify: keys are parameter names in the run_card, values are the new values
    edit_run_card(run_card_path, run_parameters)

    ## Run MG after uptate parameters and run card 
    run_command_mg5(f"launch {process_dir}\n")



    # Caminho para o arquivo de saída com eventos gerados
    output_file = os.path.join(working_dir, f"{event_name}/Events/run_01/unweighted_events.lhe")
    try:
        output_zip_file = os.path.join(working_dir, f"{event_name}/Events/run_01/unweighted_events.lhe.gz")
        subprocess.run(["gzip", "-d", output_zip_file], cwd=working_dir)
        print("Arquivo LHE descompactado com sucesso.")
    except FileNotFoundError:
        print('Arquivo ja foi descompactado')

    # Função para extrair a seção transversal (cross-section) do arquivo LHE
    def get_cross_section(lhe_file):
        cross_section = None
        with open(lhe_file, 'r') as file:
            in_init_block = False
            for line in file:
                if "<init>" in line:
                    in_init_block = True
                elif "</init>" in line:
                    in_init_block = False
                    break
                elif in_init_block:
                    data = line.strip().split()
                    if len(data) <= 6:
                        cross_section = float(data[0]) 
                        error = float(data[1])
                        break
        return cross_section, error

    # Função para ler o arquivo LHE e converter para um DataFrame do Pandas, incluindo ID do evento
    def parse_lhe_file(file_path, cross_section, error):
        events = []
        event_id = 0

        with open(file_path, 'r') as file:
            in_event = False
            for line in file:
                if "<event>" in line:
                    in_event = True
                    event = []
                    event_id += 1  # Incrementa o ID do evento a cada novo evento
                elif "</event>" in line:
                    in_event = False
                    events.append(event)
                elif in_event:
                    data = line.strip().split()
                    if len(data) >= 6:
                        try:
                            particle_data = list(map(float, data))
                            particle_data.append(event_id)  # Adiciona o ID do evento à partícula
                            event.append(particle_data)
                        except ValueError:
                            print('error in line', data)
                            pass
                                        
        # Converte a lista de eventos para um DataFrame
        columns = ["PID", "status", "Mmother1", "mother2", "color1", "color2", "px", "py", "pz", "E", "M", "Lifetime", "Spin", "Event_ID"]
        df = pd.DataFrame([item for sublist in events for item in sublist], columns=columns)
        # Adiciona uma coluna de cross-section (mesmo valor para todos os eventos)
        df["cross_section"] = cross_section #pb
        df['cross_section_error'] = error #± pb
        return df

    # Extraia a cross-section do arquivo LHE
    cross_section_value, cross_section_error = get_cross_section(output_file)

    # Parse o arquivo LHE e crie o DataFrame com a seção transversal e ID do evento
    if cross_section_value:
        df_events = parse_lhe_file(output_file, cross_section_value, cross_section_error)
        df_events = df_events.dropna(subset='E').reset_index(drop=True)
        df_events = df_events.merge(particle_df, on='PID', how='left')
        #print(df_events.head())
        df_events.to_pickle(working_dir + '/' + event_name + '/df_events.pkl')
    else:
        print("Cross-section not found in the file.")

    return df_events


In [9]:
particle_df = pd.read_csv('particle_names.csv')


In [ ]:
parameters_to_modify = {"ebeam1": 7000,
                        "ebeam2": 7000,
                        "nevents":100000,
                        #"dsqrt_shat" : 100, # minimal shat for full process,
                        #"pta":100, #       ! minimum pt for the photons 
                        #"ptamax": 1000,   #  ! maximum pt for the photons
                        #"mxx_min_pdg": "{12:2000}", # ! min invariant mass of a pair of particles X/X~ (e.g. {6:250})
                        "misset"  : 3000, #   ! minimum missing Et (sum of neutrino's momenta)
                        "missetmax": 4000, #  ! maximum missing Et (sum of neutrino's momenta)
                          } # Beam 1 energy in GeV


df_events = run_madgraph('p p > vl vl~ a', 'proton_z_met', parameters_to_modify)

In [ ]:
df_events

6.7949e-07 1.714301e-10


In [7]:
#df_events.to_csv('proton_z_met_1kk.csv')
df_events = pd.read_csv('proton_z_met_1kk.csv')

In [82]:
total_cross_section = df_events.cross_section.iloc[0] * 1000
total_cross_section_error = df_events.cross_section_error.iloc[0]
print(total_cross_section, total_cross_section_error)

1.9238e-06 6.070946e-12


In [ ]:
df_events['neutrino'] = 1
df_events['neutrino'] = -1
df_events.loc[df_events['PID'].isin([12, 14, 16, ]), 'neutrino'] = True
df_events.loc[df_events['PID'].isin([-12, -14, -16]), 'anti_neutrino'] = True
#df_events.loc[df_events['neutrino'], 'PID'] = 12
#df_events['PID_abs'] = abs(df_events['PID'])
df_gp = df_events.groupby(['Event_ID', 'neutrino', ''], as_index=False)['E'].sum()


In [ ]:
df_gp.loc[df_gp['E'] > 4000]

In [ ]:
df_to_plot = df_gp.loc[df_gp['PID'].isin([12]), 'E'].to_numpy() * total_cross_section /1000000
df_to_plot

In [ ]:

# ZZ, a pair of heavier bosons.

def get_hist_bins(df_to_plot, bin_gev):
    
    gev_bin = abs(int((df_to_plot.max() - df_to_plot.min())  / bin_gev)) # Gev
    M_hist = np.histogram(df_to_plot, bins=gev_bin) #range=(rmin, rmax))
    # the tuple `M_hist` that this function gives is so common in python that it is recognized by mplhep plotting functions
    hist, bins = M_hist  # hist=frequency ; bins=Mass values
    return hist, bins
df_to_plot = df_gp.loc[df_gp['PID'].isin([12]), 'E'].to_numpy()
hist1, bins1 = get_hist_bins(df_to_plot, 10)
df_to_plot = df_gp.loc[df_gp['PID'].isin([12]), 'E'].to_numpy()

hist1, bins1 = get_hist_bins(df_to_plot, 10)
fig, ax = plt.subplots(figsize=(10, 5))
hep.histplot(
    [hist, hist],
    bins=bins,
    histtype="fill",
    stack=True,
    color=["b", "r"],
    alpha=0.5,
    edgecolor="black",
    label=rf"""$pp \to \gamma \nu_\ell \bar \nu_\ell$ 
    $\sigma = {total_cross_section}$ fb""",
    ax=ax,
)

ax.set_xlabel(r"$\nu_l \bar \nu_l$ (MET) energy (GeV)", fontsize=15)
ax.set_ylabel(f"Events / {bin_gev} GeV", fontsize=15)
#ax.set_xlim(rmin, rmax)
ax.legend()
fig.show()


In [ ]:

# ZZ, a pair of heavier bosons.
## converts to cross section
df_to_plot = df_gp.loc[df_gp['PID'].isin([12]), 'E'].to_numpy() 
bin_gev = 0.1
#gev_bin = abs(int((df_to_plot.max() - df_to_plot.min())  / bin_gev)) # Gev
M_hist = np.histogram(df_to_plot, bins=10000) #range=(rmin, rmax))
# the tuple `M_hist` that this function gives is so common in python that it is recognized by mplhep plotting functions


hist, bins = M_hist  # hist=frequency ; bins=Mass values
fig, ax = plt.subplots(figsize=(10, 5))
hist = hist * total_cross_section /1000000
hep.histplot(
    hist,
    bins=bins,
    histtype="fill",
    color="b",
    alpha=0.5,
    edgecolor="black",
    label=rf"""$pp \to \gamma \nu_\ell \bar \nu_\ell$ 
    $\sigma = {total_cross_section}$ fb""",
    ax=ax,
)

ax.set_xlabel(r"$\nu_l \bar \nu_l$ (MET) energy (GeV)", fontsize=15)
ax.set_ylabel(f"Cross Section $\sigma$ (fb) ", fontsize=15)
ax.set_yscale('log')
ax.legend()
fig.show()


In [59]:
hist_df = pd.DataFrame(M_hist).T.rename(columns={0: 'nevents', 1: 'energy_nu'})

In [60]:
hist_df['xsec'] = hist_df['nevents'] * total_cross_section /1000000

In [62]:
hist_df['integrated_xsec'] = 0
xsec_met = {}
for energy in range(1000, 6000, 100):
    xsec_met[energy] = hist_df.loc[hist_df['energy_nu'].between(energy, energy+100), 'xsec'].sum()


In [67]:
energy_xsec_df = pd.DataFrame.from_dict(xsec_met, orient='index', columns=['xsec_met']).reset_index().rename(columns={'index': 'energy_nu'}) 

In [74]:
energy_xsec_df.to_csv('energy_met_xsec.csv', index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
hist = hist * total_cross_section /1000000
ax.plot(energy_xsec_df['energy_nu'], energy_xsec_df['xsec_met'])
ax.set_xlabel(r"$\nu_l \bar \nu_l$ (MET) energy (GeV)", fontsize=15)
ax.set_ylabel(f"Cross Section $\sigma$ (fb) ", fontsize=15)
ax.set_yscale('log')
ax.legend()
fig.show()

In [ ]:
import mplhep as hep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

hep.style.use("CMS")
# ZZ, a pair of heavier bosons.
df_to_plot = df_gp.loc[df_gp['PID'].isin([22]), 'E'].to_numpy()
df_to_plot
bin_gev = 100
gev_bin = abs(int((df_to_plot.max() - df_to_plot.min())  / bin_gev)) # Gev
M_hist = np.histogram(df_to_plot, bins=gev_bin) #range=(rmin, rmax))
# the tuple `M_hist` that this function gives is so common in python that it is recognized by mplhep plotting functions

hist, bins = M_hist  # hist=frequency ; bins=Mass values
fig, ax = plt.subplots(figsize=(10, 5))
hep.histplot(
    hist,
    bins=bins,
    histtype="fill",
    color="y",
    alpha=0.5,
    edgecolor="black",
    label=rf"""$pp \to \gamma \nu_\ell \bar \nu_\ell$ 
    $\sigma = {total_cross_section}$ fb""",
    ax=ax,
)

ax.set_xlabel(r"$\gamma$ energy (GeV)", fontsize=15)
ax.set_ylabel(f"Events / {bin_gev} GeV", fontsize=15)
#ax.set_xlim(rmin, rmax)
ax.legend()
fig.show()


In [ ]:

df_gp.loc[df_gp['PID_abs'].isin([12]), 'E'].hist(bins=50)#.set_xlim(1000, 1500)


In [ ]:
df_events.loc[df_events['PID'].isin([12, -12]), 'E'].hist(bins=50).set_xlim(1000, 1500)

In [50]:
df_events

,PID,status,Mmother1,mother2,color1,color2,px,py,pz,E,M,Lifetime,Spin,Event_ID,cross_section,cross_section_error
0,-2.0,-1.0,0.0,0.0,0.0,501.0,-0.000000,0.000000,9.715512,9.715512,0.0,0.0,1.0,1.0,13.9279,0.059495
1,2.0,-1.0,0.0,0.0,501.0,0.0,0.000000,-0.000000,-263.612678,263.612678,0.0,0.0,-1.0,1.0,13.9279,0.059495
2,22.0,1.0,1.0,2.0,0.0,0.0,9.575364,3.938082,-49.032142,50.113342,0.0,0.0,1.0,1.0,13.9279,0.059495
3,14.0,1.0,1.0,2.0,0.0,0.0,-39.046180,-25.922575,-150.143421,157.288369,0.0,0.0,-1.0,1.0,13.9279,0.059495
4,-14.0,1.0,1.0,2.0,0.0,0.0,29.470816,21.984493,-54.721602,65.926479,0.0,0.0,1.0,1.0,13.9279,0.059495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2.0,-1.0,0.0,0.0,501.0,0.0,0.000000,0.000000,270.338618,270.338618,0.0,0.0,-1.0,10000.0,13.9279,0.059495
49996,-2.0,-1.0,0.0,0.0,0.0,501.0,-0.000000,-0.000000,-17.507316,17.507316,0.0,0.0,1.0,10000.0,13.9279,0.059495
49997,22.0,1.0,1.0,2.0,0.0,0.0,19.367399,-4.487238,69.594106,72.377974,0.0,0.0,-1.0,10000.0,13.9279,0.059495
49998,16.0,1.0,1.0,2.0,0.0,0.0,21.758798,-11.736899,-2.212465,24.821263,0.0,0.0,-1.0,10000.0,13.9279,0.059495


In [ ]:
df_events.hist('E', bins=50, log=True)

In [ ]:
df_events

In [ ]:
df_events.merge(particle_df, on='PID', how='left')